In [20]:
def jaccard_index(set_1, set_2):
    #set_1, set_2 = set(item_1), set(item_2)
    u = set_1.union(set_2)
    i = set_1.intersection(set_2)
    return len(i)/len(u)

In [10]:
def get_kNN(data, X, n_neighbors, metric=jaccard_index):
    return data.map(lambda x: (x, metric(X, x['Features']))). \
        sortBy(lambda x: x[1], ascending=False). \
        take(n_neighbors)

In [3]:
import json
data = sc.textFile('hdfs://master:54310/clean'). \
    map(lambda line: json.loads(line)). \
    map(lambda doc: {'Id': doc['Id'], 'Features': doc['Text'], 'Labels': doc['Hubs']}). \
    cache()

In [4]:
word_idx = data.flatMap(lambda d: set(d['Features'])). \
    distinct(). \
    zipWithIndex(). \
    collectAsMap()

In [5]:
def vectorize_features(x: dict):
    x['Features'] = [word_idx[f] for f in x['Features']]
    return x

In [18]:
vectorized_data = data.map(lambda d: vectorize_features(d)).cache()

In [25]:
xs = data.filter(lambda d: 'Android' in d['Labels']).take(100)
xs_features = [x['Features'] for x in xs]
#xs_id = [x['Id'] for x in xs]

In [27]:
xs_vec_features = [[word_idx[f] for f in set(x_features)] for x_features in xs_features]

In [34]:
%%timeit -n1 -r 1
for x_vec_features in xs_vec_features:
    neighbors = get_kNN(vectorized_data, x_vec_features, 5)

1 loops, best of 1: 17min 25s per loop


In [36]:
x = data.filter(lambda d: 'Android' in d['Labels']).take(1)[0]
x_features = x['Features']
x_id = x['Id']
x_vec_features = [word_idx[f] for f in set(x_features)]

In [37]:
neighbors = get_kNN(vectorized_data, x_vec_features, 5)

In [38]:
ids = list(map(lambda x: x[0]['Id'], neighbors))

In [39]:
print(x_id, ids)

113549 [113549, 64214, 124237, 120883, 115577]


In [40]:
list(map(lambda x: x[1], neighbors))

[1.0,
 0.11834319526627218,
 0.10526315789473684,
 0.1044776119402985,
 0.10135135135135136]